## Tokenization using Byte-Pair Encoding and a Unigram Language Model

Author: Pierre Nugues with help from Marcus Klang

In this assignment, you will create a tokenization program to handle subwords.

In many scripts from Asia, like Chinese, Korean, or Japanese scripts, tokenization cannot rely on white spaces. The byte-pair encoding and the unigram language model are techniques that are now common in machine translation to carry out a tokenization at a subword level. Subword level tokenization shows better multilingual capabilities.

You will follow two papers: 
* Subword Regularization: _Improving Neural Network Translation Models with Multiple Subword Candidates_ by Kudo (2018) (https://arxiv.org/pdf/1804.10959.pdf) and 
* _Byte Pair Encoding is Suboptimal for Language Model Pretraining_ by Bostrom and Durrett (2020) (https://aclanthology.org/2020.findings-emnlp.414.pdf). 

In addition, you will start from a clear and easy-to-understand description in Google’s Neural Machine Translation System: _Bridging the Gap between Human and Machine Translation_ by Wu et al. (2016). (Do not read them now)
https://arxiv.org/abs/1609.08144

You will use a small corpus make it easier to test and correct your code. Note also that you will use _characters_ and not _bytes_ in this lab as this is simpler to implement. For a complete program, see the link at the end.

**In your report, be sure to answer all the questions. Please reuse the section titles of this notebook so that I can check your answers more easily**

## Preliminaries

As an overall description of the subword tokenizers, read Sections 4 (introduction paragraph) and 4.1. in the paper on translation: _Bridging the Gap between Human and Machine Translation_ by Wu et al. (2016), https://arxiv.org/abs/1609.08144.  

In your report, in a few lines (10 to 15 lines or so) you will:

1. Outline the difference with tokenization as you saw it during the course;
2. Imagine how the tokens will be learned (this will developed in the rest of the lab);
3. Summarize what could be the advantages for Asian languages, unknown words, and translation.

Commenting Sections 4 and 4.1 in your report is **mandatory**. If you are curious, you can read the complete article.

## Design of the BPE Algorithm

The first algorithm to build the subwords from a corpus is a byte-pair encoding (BPE), due to Gage (1994). In the lab, you will first read two sections of more recent articles as they are easier to understand and specifically targeted to natural language processing.

Read these two sections:

1. Section 3.1 of _Subword Regularization: Improving Neural Network Translation Models with Multiple Subword Candidates_ (https://arxiv.org/pdf/1804.10959.pdf) by Kudo (2018).
2. Section 2, algorithm 1 of _Byte Pair Encoding is Suboptimal for Language Model Pretraining_ (https://aclanthology.org/2020.findings-emnlp.414.pdf) by Bostrom and Durrett (2020).

In your report, **summarize** (10 to 15 lines or so) with your own words the byte-pair encoding (BPE) algorithm as described by Kudo (2018) and Bostrom and Durrett (2020) (Only BPE and not the unigram language model).

## BPE Programming

You will now program a byte-pair encoding program in Python. You will do it step by step. The first part will be to extract the subwords from a corpus. Note that you will use the characters, not the bytes. 

In [ ]:
import regex as re
import tqdm as tqdm
import math

First use a small corpus and then, if you have time, test your program on a larger one. Here we take the smallest novel from Selma Lagerlöf in our corpus.

In [ ]:
import os
from zipfile import ZipFile
import requests

# Parameters for Selma dataset
SELMA_URL = "https://github.com/pnugues/ilppp/raw/master/programs/corpus/Selma.zip"

SELMA_FILES = [
    os.path.join("Selma", fname) 
    for fname in 
    [
        "bannlyst.txt", 
        "gosta.txt", 
        "herrgard.txt", 
        "jerusalem.txt", 
        "kejsaren.txt", 
        "marbacka.txt", 
        "nils.txt", 
        "osynliga.txt", 
        "troll.txt"
    ]
]

def download_and_extract_selma():
    """Downloads and unpacks Selma.zip"""
    
    # Download if not all files exist
    req = requests.get(SELMA_URL, stream=True)
    if req.status_code != 200:
        print("Failed to download file, got status: " + req.status_code)
        req.close()
    else:
        with open("Selma.zip", "wb") as fd:
            written = 0
            for chunk in req.iter_content(chunk_size=65536):
                fd.write(chunk)
                written += len(chunk)
                print("Downloading: %d bytes written to Selma.zip" % written)

        print("Selma.zip donwnloaded.")
        req.close()
        
        selma_zipfile = ZipFile("Selma.zip")
        selma_files_to_extract = [zi for zi in selma_zipfile.filelist if not zi.filename.startswith("__") and zi.filename.endswith(".txt")]
        for zi in selma_files_to_extract:
            selma_zipfile.extract(zi)
            print("Extracted: " + zi.filename)
            
        print("Done!")
        
# If not all path exists (all are true), then download
if not all([os.path.exists(fname) for fname in SELMA_FILES]):
    download_and_extract_selma()
else:
    print("Selma has been downloaded.")
    
SELMA_FILES

In [ ]:
#FILE_PATH = '../../corpus/Selma.txt'
FILE_PATH = 'Selma/herrgard.txt'

Read the corpus and store it in the `corpus` string variable.

In [ ]:
with open(FILE_PATH, encoding='utf8') as f:
    corpus = f.read().strip()

Replace all the space sequences in `corpus`, including newlines and tabulations, and normalize them as one space.

In [ ]:
# Write your code
corpus = re.sub('\p{Whitespace}+', " ", corpus)

In [ ]:
corpus[:100]

### BPE

#### Initial Vocabulary

Write the code (one instruction) to split the corpus in a list of characters and store the results in `corpus_l`. This is just a type conversion. Given the input:
<pre><span style="font-size: 12pt;">corpus = 'De senaste fem &aring;ren har cirka 25 000 unga'</span></pre>

Return:
<pre><span style="font-size: 12pt;">corpus_l = ['D', 'e', ' ', 's', 'e', 'n', 'a', 's', 't', 'e', ' ', 'f', 'e', 'm', ' ', ...]</span></pre>

In [ ]:
# Write your code
corpus_l = list(corpus)

In [ ]:
corpus_l[:15]

Extract the set of characters that will serve as initial subword tokens:

1. Write a statement to extract the set of all the characters from `corpus_l`; 
2. Exclude the space from this set and call the resulting set: `char_set`.

In [ ]:
# Write your code
char_set = set(corpus_l) - set(' ')

In [ ]:
len(char_set)

Using code from the previous question, write an `initial_vocabulary()` function taking the the `corpus_l` variable as input and returning the the set of all characters appearing in the corpus (the initial character set), deprived from the white space.

In [ ]:
# Write your code here
def initial_vocabulary(corpus_l):
    return set(corpus_l) - set(' ')

In [ ]:
initial_vocabulary(corpus_l)

#### Counting

Write a `pair_count()` function that takes a list of tokens as input, possibly single characters or subword tokens, and that counts the adjacent pairs (bigrams). You will implement these counts as dictionaries: The key will be a pair (tuple) of adjacent symbols and the value, its frequency. Remember that you cannot cross whitespaces, i.e. a pair cannot include a whitespace.

Given the input

`['D', 'e', ' ', 's', 'e', 'n', 'a', 's', 't', ...]`
count_pairs should return a dictionary: 


`{('D', 'e'): 1, ('s', 'e'): 1, ('e', 'n'): 1, ('n', 'a'): 1, ...}`

In [ ]:
# Write your code here
from collections import Counter
def pair_count(corpus_l):
    pairs = []
    for c1, c2 in zip(corpus_l[:-1], corpus_l[1:]):
        if c1 == ' ' or c2 == ' ':
            continue
        pairs += [(c1, c2)]

    return dict(Counter(pairs))

In [ ]:
pairs = pair_count(corpus_l)

In [ ]:
# pairs

Determine the most frequent pair

In [ ]:
# write your code
most_freq_pair = max(pairs, key= pairs.get)

In [ ]:
most_freq_pair

In [ ]:
''.join(most_freq_pair)

#### The First Iteration

We store the initial symbols in a `vocabulary` variable

In [ ]:
vocabulary = initial_vocabulary(corpus_l)

In [ ]:
len(vocabulary)

Add your most frequent pair to the vocabulary after one iteration

In [ ]:
# write your code here
def add_to_voc(corpus_l):
    pairs = pair_count(corpus_l)
    most_freq_pair = ''.join(max(pairs, key= pairs.get))
    return most_freq_pair

extended_voc = vocabulary.union({add_to_voc(corpus_l)})

In [ ]:
extended_voc

In [ ]:
len(extended_voc)

#### Incremental Construction
We will now incrementally build the vocabulary.

Create a `merge_bigrams()` function that takes a list of tokens, `corpus_l`, and a pair of subword tokens `(token_r, token_l)` as input and merges adjacent sequences token_r, token_l into a new token, `token_new`, replacing the sequence `token_r, token_l` in `corpus_l`. Your function will return a new list. 

Given the input 

`corpus_l = ['D', 'e', ' ', 's', 'e', 'n', 'a', 's', 't', ...]`

`merge_bigrams(corpus_l, ('e', 'n'))` should return where all the seuquences of 'e' and 'n' have been merged:

`['D', 'e', ' ', 's', 'en', 'a', 's', 't', ...]`

And reapplying `merge_bigrams(corpus_l, ('s', 'en'))` to this corpus should return

`['D', 'e', ' ', 'sen', 'a', 's', 't', ...]`

You will apply a greedy algorithm. Given the pair ('a', 'a') and the list ['a', 'a', 'a'], the result will be: ['aa', 'a']

In [ ]:
# Write your code here
class Remove:
    pass

def merge_bigrams(corpus_l, pair):
    new_corpus = corpus_l.copy()
    skip_index = -1
    for index, this_pair in enumerate(zip(corpus_l[:-1], corpus_l[1:])):
        if this_pair == pair:
            new_corpus[index] = ''.join(this_pair)
            new_corpus[index + 1] = "REMOVEME"
        
    return [element for element in new_corpus if  element != "REMOVEME"]

In [ ]:
corpus_test = ['D', 'e', ' ', 's', 'e', 'n', 'a', 's', 't']
merge_bigrams(corpus_test, ('e', 'n'))

In [ ]:
merge_bigrams(merge_bigrams(corpus_test, ('e', 'n')), ('s', 'en'))

#### Byte Pair Encoding (BPE): Building the Vocabulary

Write now a `BPE()` function following Algorithm 1 in _Byte Pair Encoding is Suboptimal for Language Model Pretraining_ by Bostrom and Durrett (2020). 

Your function will take `corpus_l` and the vocabulary size `k` as input. This size `k` will correspond to the count of new subwords added to the initial list of symbols. With your initial corpus, you should have 67 found symbols. With `k = 10`, you will add 10 subwords to this initial list. Note that Bostrom and Durrett (2020) define their $k_\text{Bostrom and Durrett}$ as `k + initial vocabulary`. 

Return the vocabulary of subword tokens in the form of a list: the initial vocabulary and the subwords you will create.

You will start from the initial vocabulary and `k` will be the number of symbols you add to this vocabulary.

In [ ]:
# Write your code here
def BPE(corpus_l, k=10, verbose=True):
    vocabulary = initial_vocabulary(corpus_l)
    print(len(vocabulary))
    
    corpus = corpus_l.copy()
    for _ in tqdm.tqdm(range(k)):
        # find most common bigram
        pair_counts = pair_count(corpus)
        most_freq_pair = max(pair_counts, key=pair_counts.get)
        
        # update corpus
        corpus = merge_bigrams(corpus, most_freq_pair)
        # update vocabulary
        new_token = ''.join(most_freq_pair)
        vocabulary = vocabulary.union({new_token})
        print(new_token, end=' ') if verbose else None
    
    del corpus
    return vocabulary

In [ ]:
len(corpus_l)

We build a vocabulary of 50 subwords in addition to our initial set of symbols

In [ ]:
vocabulary = BPE(corpus_l, 50)
vocabulary

In [ ]:
len(vocabulary)

#### BPE Tokenizer

You will now use the vocabulary you obtained to tokenize a text stored in the corpus string.

You will implement a greedy technique building on Python's regular expression engine. You will call this function `tokenize_bpe()` that will take two inputs: `corpus` and `vocabulary`, and that will return the tokenized text in the form of a list.

    def tokenize_bpe(corpus, vocabulary):

      ...

      return tokens
Here are a few hints on how to write this function. Before you call a regular expression and apply it to a text, a regex engine compiles it into an efficient automaton (you do not need to call `compile()` as the automaton is automatically cached). The only thing you have to take care of is the length order of the strings. In the tokenization function:

1. Write a statement to order the strings in your vocabulary list,
  * first by decreasing length, and then
  * by alphabetic order.
  
  You will call this list `vocabulary_srt`; Knowing that, in the ASCII order, the upper case letters are placed before lower case ones, the list: ['D', 'e', 'sen', 'a', 's', 't']

will be sorted as: ['sen', 'D', 'a', 'e', 's', 't']

2. Escape the regular expression with `re.escape()` as some strings may include metacharacters, for instance 'a.', where the dot matches all the characters.
3. Convert this list into a regular expression that results in a disjunction of subword tokens. Remember that the disjunction operator (or) for regular expressions is the vertical bar (`|`), as in `'a'|'b'`, meaning match `'a'` or `'b'`;
3. Apply a regular expression function to tokenize your text: the corpus string. You will use `findall()`for this. You will return this result.

In [ ]:
re.escape('a.')

In [ ]:
# Write your code here
from functools import reduce 
def tokenize_bpe(corpus, vocabulary):
    # sort tokens
    v = list(vocabulary).copy()
    v.sort(key=lambda x: (-len(x), x))
    
    pattern = reduce(lambda x, y: f'{x}|{re.escape(y)}', v)
    return list(re.findall(pattern, corpus))

In [ ]:
print(tokenize_bpe(corpus, vocabulary)[:200])

## Unigram Language Model

You are now done with BPE and you can now consider the unigram language model.

Read these two sections:

1. Section 3.2 of _Subword Regularization: Improving Neural Network Translation Models with Multiple Subword Candidates_ (https://arxiv.org/pdf/1804.10959.pdf) by Kudo (2018).
2. Section 2, algorithm 2 and the related text of _Byte Pair Encoding is Suboptimal for Language Model Pretraining_ (https://aclanthology.org/2020.findings-emnlp.414.pdf) by Bostrom and Durrett (2020).

In your report, **summarize** (10 to 15 lines or so) with your own words the tokenization with a unigram language model as described by by Kudo (2018) and Bostrom and Durrett (2020). You will notably consider two aspects:
1. How to obtain the subword vocabulary;
2. How to tokenize a text.

In your report, given what you have done on the byte-pair encoding, how would you build the “reasonably big seed vocabulary” needed for the unigram language model?

### Unigram Probabilities

Starting from the “reasonably big seed vocabulary”, you will now fit a unigram language model. You will start with a vocabulary of 50 subwords in addition to the character set and reduce it to 49, i.e. you will find one subword to discard.

Kudo (2018) proposes the expectation-maximization algorithm that we have not seen in the course on natural language processing. Instead, in this lab, you will approximate the language model with the BPE algorithm.

Write a `unigram_lm()` function that takes a corpus string and a vocabulary of subword tokens as input and returns a dictionary, where the keys are the subwords and each key value, the key relative frequency:

    def unigram_lm(corpus, vocabulary):

       ...

      return unigram_probs
Your function will:

1. Tokenize your corpus with BPE (you can reuse the `tokenize_bpe()` function);
2. Estimate the probability of each word (simply count the occurrences of the subwords and divide them by the length of the tokenized corpus);
3. Return this model as a dictionary.

In [ ]:
# Write your code here
def unigram_lm(corpus, vocabulary):
    tokenized_corpus = tokenize_bpe(corpus, vocabulary)

    counter = Counter(tokenized_corpus)
    
    n = len(tokenized_corpus)
    unigram_probs = {token: c / n for token, c in counter.items()}
    return unigram_probs

In [ ]:
unigram_probs = unigram_lm(corpus, vocabulary)
unigram_probs

In [ ]:
len(unigram_probs)

### Unigram Tokenization

You will now apply your unigram language model to tokenize a character sequence that does not include spaces, typically a single word in the Latin or Greek scripts or a sequence of words in Asian scripts, like Chinese or Korean.

Write a `tokenize_lm()` function that takes a character sequence, `char_seq`, and a dictionary of unigram probabilities, `unigram_probs`,  as input and returns the subword tokens and the segmentation probability, (prob,tokens). You will only return the token list with the highest probability.

    def tokenize_lm(char_seq, unigram_probs):

      ...

      return max(candidates)

As an example, applying 

tokenize_lm('senare', unigram_probs)
results in

`(2.0899522820189735e-07, ['s', 'en', ar', 'e'])`

Your function will cache (memoize) the results to speed up the computation. It will be similar to that of Norvig's in the notebook: How to Do Things with Words.ipynb. You can reuse it.
Python has a built-in memoization function that you can use: @functools.lru_cache(maxsize=2**10). You can also use the newer @functools.cache() function if you have Python 3.9 or higher. See here: https://docs.python.org/3/library/functools.html.

In [ ]:
"""
from typing import List, Tuple

def tokenize_lm(char_seq, unigram_probs):
    
    @functools.cache # Available from Python 3.9
    def dp(i: int, progress: tuple) -> Tuple[Tuple[str], float]:
        
        if i >= n  : return 1, tuple()
        if i == n-1: 
            progress = (char_seq[i], )
            print("base case", i, char_seq[i], progress)
            return unigram_probs[char_seq[i]], progress
        
        dp1, best_seq1 = dp(-1, progress)
        dp2, best_seq2 = dp(-1, progress[1:]) 
        
        print("at", char_seq[i])
        print(f"{best_seq1=}")
        print(f"{best_seq2=}")

        this_token = char_seq[i]
        print(f"{this_token=}")
        
        next_token = progress[0]
        print(f"{next_token=}")
        alt1 = unigram_probs[this_token] * dp1 if this_token in unigram_probs else float('-inf')
        alt2 = unigram_probs[merged_token] * dp2 if (merged_token := this_token + next_token) in unigram_probs else float('-inf')        
        
        maxprob, best_sentence = (
            (alt1, (this_token, ) + best_seq1) 
            if alt1 >= alt2 
            else (alt2, (this_token + next_token, ) + best_seq2)
        )
        print("alt1", (this_token, ) + best_seq1)
        print("alt2", (this_token + next_token, ) + best_seq2) 
        print("best_sentence", best_sentence)
        
        progress = best_sentence 
        print("progress", progress)
                      
        return maxprob, best_sentence
    
    progress = tuple()   
    n = len(char_seq)
    return dp(0, progress)
"""
"""
    dp(i):
        maximal probability of subword token in char_seq[i:]

    compute:
        dp(0)

    base cases:
        one char left: return unigram_probs of this token
        outside: return 1


    recursive step:
        max(
            prob(leave this token alone) + prob(rest),
            prob(merge it with the one to the right) + prob(rest + 1 skip merged token)
        )

    a tt
    
    prob(att) * 1
    
    senare
    ... a, r, e, ...
    OR
    ... ar, e, ...

    r, e
    r + e
    re + 0
    """


In [ ]:
"""
from typing import List, Tuple

def tokenize_lm(char_seq, unigram_probs):
    
    @functools.cache # Available from Python 3.9
    def dp(i: int, progress: tuple) -> Tuple[float, Tuple[str]]:
        
        if i >= n  : return 1, tuple()
        if i == n-1: 
            progress = tuple(char_seq[i]) + progress
            #print("base case", i, char_seq[i], progress)
            return unigram_probs[char_seq[i]], progress
        
        dp1, best_seq1 = dp(i+1, progress)
        dp2, best_seq2 = dp(i+2, progress) 
        
        #print("at", char_seq[i])
        #print(f"{best_seq1=}")
        #print(f"{best_seq2=}")

        this_token = char_seq[i]
        #print(f"{this_token=}")
        next_token = best_seq1[0]
        
        alt1 = unigram_probs[this_token] * dp1 if this_token in unigram_probs else float('-inf')
        alt2 = unigram_probs[merged_token] * dp2 if (merged_token := this_token + next_token) in unigram_probs else float('-inf')        
        
        maxprob, best_sentence = (
            (alt1, (this_token, ) + best_seq1) 
            if alt1 >= alt2 
            else (alt2, (this_token + next_token, ) + best_seq2)
        )
        #print("alt1", (this_token, ) + best_seq1)
        #print("alt2", (this_token + next_token, ) + best_seq2) 
        #print("best_sentence", best_sentence)
        
        progress = best_sentence 
        #print("progress", progress)
                      
        return maxprob, progress
    
    progress = tuple()   
    n = len(char_seq)
    return dp(0, progress)
    """
    """
    dp(i):
        maximal probability of subword token in char_seq[i:]

    compute:
        dp(0)

    base cases:
        one char left: return unigram_probs of this token
        outside: return 1


    recursive step:
        max(
            prob(leave this token alone) + prob(rest),
            prob(merge it with the one to the right) + prob(rest + 1 skip merged token)
        )
"""

In [ ]:
import functools
import numpy as np
def splits_(text, start=0, L=20):
    "Return a list of all (first, rest) pairs; start <= len(first) <= L."
    return [(text[:i], text[i:]) 
            for i in range(start, min(len(text), L)+1)]


def segment(text, probs):
    @functools.lru_cache(maxsize=2**10)
    def _segment(text):
        if not text: 
            return []
        else:
            candidates = ([first] + _segment(rest)
                        for (first, rest) in splits_(text, 1))
            return max(candidates, key=lambda x: Pwords(x, probs))
        return _segment(text)


def Pword(w, probs):
    return 0 if w not in probs else probs[w] 

def Pwords(words, probs):
    "Probability of words, assuming each word is independent of others."
    return product(Pword(w, probs) for w in words)

def product(nums):
    "Multiply the numbers together.  (Like `sum`, but with multiplication.)"
    result = 1
    for x in nums:
        result *= x
    return result

def tokenize_lm(char_seq, probs):
    # Use one of the two cache functions below to have a faster answer:  
    
    #@functools.cache # Available from Python 3.9
    # The arguments of the cached function must be hashable that's why we define an inner cacheable function
    @functools.lru_cache(maxsize=2**10)
    def __tokenize_lm(char_seq):
    # Write your code here
        sequence = segment(char_seq, probs)
        p = Pwords(sequence, probs)
        return (p,  sequence) 
    
    return __tokenize_lm(char_seq)

In [ ]:
%%time
tokenize_lm('senare', unigram_probs)

In [ ]:
tokenize_lm('att', unigram_probs)

In [ ]:
unigram_probs["att"], unigram_probs["a"], unigram_probs["tt"], unigram_probs["a"] * unigram_probs["tt"]

### Text Tokenization with Unigrams

The previous function applies to a sequence without spaces. You will now apply it to your corpus. Write a `tokenize_text_lm()` function that takes the whole `corpus` string as input and the unigram probabilities `unigram_probs` and return the corpus probability and the tokenized subwords. 

This function is just an application of the functions you just wrote, where you will:
1. `split()` the string by whitespaces
2. Break the tokens into subtokens and compute the probabilities of the resulting sequences;
3. Sum the logarithm of these probabilities. Use log10 to check your output with the numbers in the notebook. 

It is very significant that you use the logarithm of the probabilities and the sum. If you multiply the probabilities, you will get an underflow.

In [ ]:
# Write your code
def tokenize_text_lm(corpus, unigram_probs):
    words = corpus.split(' ')
    
    tokens = []
    log_probs = 0
    i = 0
    for word in tqdm.tqdm(words):
        prob, word_tokens = tokenize_lm(word, unigram_probs)
        
        log_probs += np.log10(prob)
        tokens += word_tokens
        
        if i % 1000 == 0:
            print(f"{i=}, {log_probs=:.3f}, {word_tokens=}, {np.log10(prob)=:.3f}")
        i += 1

    
    return log_probs, tokens

In [ ]:
init_loglikelihood, tokens = tokenize_text_lm(corpus, unigram_probs)

In [ ]:
init_loglikelihood, tokens[:10]

### Vocabulary Selection

You will now implement the final loop, where you will, at each iteration:
1. Select one subword from the vocabulary.
2. Compute the resulting log-likelihood of the corpus without this word.
3. Compute the loss, i.e. the log-likelihood reduction when the subword is removed from the current vocabulary

You will always keep the single characters in your vocabulary to avoid unknown words.

Store the pairs, (log-likelihood, removed_subword) in a list `logloss_word` and rank them by likelihood value.

In [ ]:
# Write your code here
def compute_logloss(corpus, vocabulary, init_loglikelihood):
    logloss_word = []
    
    _vocabulary = vocabulary.copy()

    for removed_token in _vocabulary:
        if len(removed_token) == 1: continue
            
        new_vocabulary = vocabulary.copy()
        new_vocabulary.remove(removed_token)
        
        #print(f"{removed_token=}")
        #print(f"{(removed_token in new_vocabulary)=}")

        _unigram_probs = unigram_lm(corpus, new_vocabulary)
        #print(f"{removed_token=}")
        logloss, _ = tokenize_text_lm(corpus, _unigram_probs)
        logloss_word.append((logloss, removed_token))
        #print(f"{(logloss, removed_token)=}")
        
    _ = [(logloss - init_loglikelihood, word) for logloss, word in logloss_word]
    return sorted(_)

In [ ]:
logloss_word = compute_logloss(corpus, vocabulary, init_loglikelihood)

You will reduce now your vocabulary by one token: `out_candidate`. Write the piece of code to determine it.

In [ ]:
# Write your code here
out_candidate = min(logloss_word, key=lambda x: logloss_word[0])
print(f"{logloss_word=}, {out_candidate=}")

In [ ]:
out_candidate = (-92.75720750979963, 'tt')

If you are interested, you can improve this program and test it on larger corpora. You can also read a fine implementation of BPE by Andrej Karpathy: https://github.com/karpathy/minGPT/blob/master/mingpt/bpe.py

## Submission

When you have written all the code and run all the cells, fill in your ID and as well as the name of the notebook.

In [ ]:
STIL_ID = ["si7660da-s", "ni5324ro-s"] # Write your stil ids as a list
CURRENT_NOTEBOOK_PATH = os.path.join(os.getcwd(), 
                                     "5-BPE_solution.ipynb") # Write the name of your notebook

The submission code will send your answer. It consists of the subword to discard.

In [ ]:
import json
ANSWER = json.dumps({'out_candidate': out_candidate})
ANSWER

Now the moment of truth:
1. Save your notebook and
2. Run the cells below

In [ ]:
SUBMISSION_NOTEBOOK_PATH = CURRENT_NOTEBOOK_PATH + ".submission.bz2"

In [ ]:
import bz2
ASSIGNMENT = 5
API_KEY = "f581ba347babfea0b8f2c74a3a6776a7"

# Copy and compress current notebook
with bz2.open(SUBMISSION_NOTEBOOK_PATH, mode="wb") as fout:
    with open(CURRENT_NOTEBOOK_PATH, "rb") as fin:
        fout.write(fin.read())

In [ ]:
res = requests.post("https://vilde.cs.lth.se/edan20checker/submit", 
                    files={"notebook_file": open(SUBMISSION_NOTEBOOK_PATH, "rb")}, 
                    data={
                        "stil_id": STIL_ID,
                        "assignment": ASSIGNMENT,
                        "answer": ANSWER,
                        "api_key": API_KEY,
                    },
               verify=True)

# from IPython.display import display, JSON
res.json()

## Turning in your assignment

Now your are done with the program. To complete this assignment, you will write a report where you will:
1. Describe the background as well as the algorithms you used. For this, summarize the articles as described in the notebook:
   * Preliminaries: subword tokenizers
   * Design of the BPE Algorithm
   * Unigram Language Model
2. Describe your program as well as your results

The whole report should be of 2 to 3 pages.

Submit your report as well as your **notebook** (for archiving purposes) to Canvas: https://canvas.education.lu.se/. To write your report, you can either
1. Write directly your text in Canvas, or
2. Use Latex and Overleaf (www.overleaf.com). This will probably help you structure your text. You will then upload a PDF file in Canvas.

The submission deadline is October 14, 2022.

## Curious?

If you are interested, you can improve this program and test it on larger corpora. You can also read a fine implementation of BPE by Andrej Karpathy: https://github.com/karpathy/minGPT/blob/master/mingpt/bpe.py